In [19]:
import pandas as pd
from sklearn.svm import SVR

Different simple models
1. Weekly median from all time
2. Monthly median from all time
3. Monthly median from last year
4. Last year's value
6. Lin Reg for all years on store, dept, week

In [8]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
features = pd.read_csv("features.csv")
stores = pd.read_csv("stores.csv")

In [9]:
# feature engineering for all
def add_feautres(target, stores, features):
    temp = target.merge(stores, on="Store").merge(features, on=["Store", "Date"])
    temp.drop("IsHoliday_y", axis=1, inplace=True)
    temp.drop("Size", axis=1, inplace=True)
    temp.drop("Temperature", axis=1, inplace=True)
    temp.drop("Type", axis=1, inplace=True)
    temp.fillna(0, inplace=True)
    temp["Date"] = pd.to_datetime(temp.Date)
    temp["year"] = temp.Date.dt.year
    temp["week"] = temp.Date.dt.week
    temp["month"] = temp.Date.dt.month
    
    return temp

train = add_feautres(train, stores, features)
test = add_feautres(test, stores, features)

In [10]:
def make_dept_weekly_median(df):
    """ takes a training df and returns a df with the store, dept, week, 
        and avg weekly sales"""
    gb = df.groupby(["Store", "Dept", "week"]).median()["Weekly_Sales"]
    gb.rename({"Weekly_Sales": "average_weekly_sales"})
    return gb

def make_dept_monthly_median(df):
    """ takes a training df and returns a df with the store, dept, week, 
        and avg monthly sales"""
    gb = df.groupby(["Store", "Dept", "month"]).median()["Weekly_Sales"]
    gb.rename({"Weekly_Sales": "average_monthly_sales"})
    return gb

def make_last_record(df):
    """ Gets the latest value for each of the store, dept, week combinations"""
    gb_idx = df.groupby(["Store", "Dept", "week"])['year'].transform(max) == df['year']
    prev_year = df[gb_idx][["Store", "Dept", "week", 'Weekly_Sales', "month"]]
    return prev_year

In [11]:
def predict_from_median(means_gb, target_df, time):
    return target_df.join(means_gb, on=["Store", "Dept", time], rsuffix= "_"+time+"_means")

def add_prev_year(prev_year_gb, target_df):
    return target_df.join(prev_year_gb, on=["Store", "Dept", "week"], rsuffix="_pyear")

In [12]:
prev_year = make_last_record(train)
monthly_medians = make_dept_monthly_median(prev_year)
weekly_medians = make_dept_weekly_median(train)

In [ ]:
models = {}
for store in train.Store.unique():
    for dept in train.Dept.unique():
        for week in train.week.unique():
            try:
                key = f"{store}_{dept}_{week}"

                years = train[(train["Store"] == store) & 
                              (train["Dept"] == dept) & 
                              (train["week"] == week)]["year"]
                values = train[(train["Store"] == store) & 
                               (train["Dept"] == dept) & 
                               (train["week"] == week)]["Weekly_Sales"]

                svr_rbf = LinearSVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
                svr_rbf.fit(years.values.reshape(-1,1), values)
                models[key] = svr_rbf
            except:
                print(key)
                print("\t",years)
                print("\t",values)
                print()
    

In [22]:
models = {}
for store in train.Store.unique():
    for dept in train.Dept.unique():
        for week in train.week.unique():
            key = f"{store}_{dept}_{week}"

            years = train[(train["Store"] == store) & 
                          (train["Dept"] == dept) & 
                          (train["week"] == week)]["year"]
            values = train[(train["Store"] == store) & 
                           (train["Dept"] == dept) & 
                           (train["week"] == week)]["Weekly_Sales"]

            plt.plot()
            models[key] = svr_rbf

-0.00398853766622409
-0.34071954759826517
-0.0059490184936781265
-0.2987104201562283
-0.25208739474747466
0.019270443065507403
-0.34718132279638936
-0.1629016953607818
-0.17250398190627858
-0.010820591480210862
-0.053400685799373404
-0.4649024990433668
-0.48926335636064544
-0.37837742339262115
0.009657769736210242
-0.3271114335081522
-0.2500418837056022
-0.07767316210635089
0.012760890809824232
0.01473663536534564
-0.08064834256431719
0.01653355294624903
-0.4006643769260338
-0.020849872069058195
0.10693946748720162
0.05382737414078553
-0.01961570668642043
-0.03717262054244408
-0.006439962097784412
-0.12194008786640498
-0.08880608988247407
-0.21648738095466946
-0.1732884920133697
-0.1682065498580112
-0.024238146173695796
0.0054442583603765415
-0.364058612273358
-0.01087842161635888
-0.03108285151382173
0.00672917953443275
0.04377944276860957
0.07436318534552532
0.018121720823896403
0.013665524987100386
0.020939743818588097
0.04376333816403277
0.004159200951419528
0.008986423743130412
0.

0.04693299296733544
0.1113836243541445
0.1834124450859853
0.18621352411782233
0.04651439211774688
0.07993254878233248
-0.10413782704889396
-0.3312352919613308
-0.14884241346599425
-0.2932790730124737
0.017644393322999274
-0.22458722299624845
-0.32873700634850533
-0.029219996259613575
-0.3207906080261673
-0.35074889662564757
-0.3812625542783538
-0.27696177896260377
-0.3831203351870396
0.004224335191908013
-0.0039822522799448645
-0.010187856645392124
-0.3144995993584214
-0.16924745430581956
-0.27921162239286934
-0.14443368712140137
-0.42481832454857926
-0.05710053708516849
-0.07158701336463769
-0.002755913832766632
-0.03034276637793254
-0.2578398937306925
0.027950427231446806
-0.4328569203480528
0.0024367350031643253
-0.19610241812779816
-0.05189431025063218
-0.04112914160761871
-0.13413230783688923
0.016551963529840785
-0.16247610378376165
-0.1767747907858841
-0.37100523176937084
-0.3070778254854638
-0.04905695846520075
0.02013038067381523
0.07429644388308565
0.02757171357127919
0.01665

0.061192042423311443
-0.018303656543469993
0.02538028544898918
0.02935696120638587
0.07080536740345555
0.037403111907336806
0.024324805398285076
0.026387508149273486
0.02052216284175623
0.02741820417912877
0.19003326208456328
0.07530584360968273
0.03963487224060458
0.11902849207422672
0.07661798827678779
-0.09742588512108386
-0.08083372608518968
0.013565184058671353
-0.027161937557925286
0.02923134694030105
-0.350460620217091
-0.06702896828604987
0.010768809365734433
-0.30404082356313755
-0.001275815872156194
-0.2111527426678219
-0.4994027541301709
0.004762136665572236
0.05489616579591683
-0.4644861570318055
nan
0.9847584470987062
0.9973701476720066
0.9977241803631257
nan


C:\Users\admin7\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\admin7\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.